In [1]:
from scipy.sparse import load_npz
from igraph import Graph
import numpy as np
import pandas as pd

In [2]:
threshold = 0.5
subset_adducts = True

In [3]:
DATA_DIR = "../data/"

# Read in data

In [4]:
# File paths
matrix_path = f"{DATA_DIR}/processed/entropy_scores_matrix.npz"
metadata_path = f"{DATA_DIR}/processed/entropy_scores_metadata.npz"

# Load the sparse matrix
combined_sparse_matrix = load_npz(matrix_path)

# Make sure matrix is symmetric (there are sometimes same floating point differences)
combined_sparse_matrix = combined_sparse_matrix.maximum(
    combined_sparse_matrix.transpose()
)

In [5]:
# Load row and column names
metadata = np.load(metadata_path, allow_pickle=True)
row_names = metadata["row_names"]
col_names = metadata["col_names"]

# Subset data to only [M+H]+ adducts

In [6]:
if subset_adducts:
    cleaned_spectral_data = pd.read_parquet(
        f"{DATA_DIR}/processed/enpkg_with_ms_data_cleaned.pq"
    )

    cleaned_spectral_data = cleaned_spectral_data[
        cleaned_spectral_data["adduct"] == "[M+H]+"
    ]

    row_name_to_index = {name: i for i, name in enumerate(row_names)}
    col_name_to_index = {name: i for i, name in enumerate(col_names)}
    subset_row_indices = [
        row_name_to_index[name] for name in cleaned_spectral_data["row_uid"]
    ]
    subset_col_indices = [
        col_name_to_index[name] for name in cleaned_spectral_data["row_uid"]
    ]

    combined_sparse_matrix = combined_sparse_matrix[subset_row_indices, :][
        :, subset_col_indices
    ]
    row_names = row_names[subset_row_indices]
    col_names = col_names[subset_col_indices]
    print(combined_sparse_matrix.shape)

(335137, 335137)


# Create graph

In [7]:
# Filter out values below the threshold
combined_sparse_matrix = combined_sparse_matrix.multiply(
    combined_sparse_matrix > threshold
)

In [8]:
# Get edges
sources, targets = combined_sparse_matrix.nonzero()
edges = list(zip(sources, targets))

In [9]:
# Create graph
g = Graph(edges=edges, directed=False)

In [10]:
# Get connected components
components = g.connected_components()

In [11]:
print("Num clusters: ", len(components))

Num clusters:  41931


# Map back to row uids

In [12]:
# Create cluster labels
membership = components.membership
labels = {c: m for c, m in zip(row_names, membership)}

In [13]:
df = pd.read_parquet(f"{DATA_DIR}/processed/enpkg_with_ms_data_cleaned.pq")
df["cluster"] = df["row_uid"].map(labels)

In [13]:
# save cluster labels
df.to_parquet(
    f"{DATA_DIR}/clustering_with_networks/spectral_entropy_10ppm_{threshold}.pq"
)

# Make sure clusters behave as expected

In [14]:
df["precursor_diff"] = df.groupby("cluster")["ms2_precursor_mz"].transform(
    lambda x: max(x) - min(x)
)

In [15]:
# These should all be close to 0
df.precursor_diff.describe()

count    335137.000000
mean          0.004734
std           0.003043
min           0.000000
25%           0.002500
50%           0.004800
75%           0.006800
max           0.017800
Name: precursor_diff, dtype: float64